<a href="https://colab.research.google.com/github/hniu-tamu/cotton_water_stress_remote_sensing/blob/main/A_parameter_tuning_demo_twri_rgb_uav_cnn_train_four_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import sklearn
import time
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import sys
import numpy
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
numpy.set_printoptions(threshold=sys.maxsize)

In [3]:
pwd

'/content'

In [7]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [5]:
pwd

'/content'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "CNN images"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [9]:

def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],  # minus 1 because 1,2,3,4
                                100*np.max(predictions_array),
                                class_names[true_label]), # minus 1 because 1,2,3,4
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(4))
  plt.yticks([])
  thisplot = plt.bar(range(4), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')



In [ ]:
pip install scikeras==0.1.8

In [66]:
def create_model(neurons=64, kernel_size=(3, 3), pool_size=(2, 2), optimizer='adam'):
      model = models.Sequential()
      model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
      model.add(layers.MaxPooling2D(pool_size))
      model.add(layers.Conv2D(neurons, kernel_size, activation='relu'))
      model.add(layers.MaxPooling2D(pool_size))
      model.add(layers.Conv2D(neurons, kernel_size, activation='relu'))
      model.add(layers.Flatten())
      model.add(layers.Dense(neurons, activation='relu'))
      model.add(layers.Dense(4))
      # model.summary()
      # compile the model
      model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
      return model

In [ ]:
for date in ["8_18"]: # "8_18", "9_02", "9_09", "9_20"
  for img_size in [64]: # 32, 64, 128
    # load the data
    data = np.load(f'/content/drive/MyDrive/Colab Notebooks/twri_rgb_6832_cotton_{img_size}x{img_size}_{date}.npy')
    labels = pd.read_csv('labels_rgb.csv')
    y_label = labels['class'].values

    # remove the filler type
    new_data_without_filler = np.zeros((5376, img_size, img_size, 3)) # defult was 32,32,3
    new_y_label_without_filler = np.zeros(5376, dtype=int)
    i = 0
    for idx, val in enumerate(y_label):
      if val != 0:
        new_data_without_filler[i] = data[idx]
        new_y_label_without_filler[i] = y_label[idx]
        i+=1
    data = new_data_without_filler
    y_label = new_y_label_without_filler - 1

    # split the data and normalization
    train_images, test_images, train_labels, test_labels = train_test_split(data, y_label, test_size=0.3, random_state=42)
    train_images, test_images = train_images / 255.0, test_images / 255.0
    class_names = ['rainfed', 'fully irrigated', 'percent deficit', 'time delay']

    # Define the hyperparameters to search
    param_grid = {
      'optimizer': ['adam', 'sgd'],
      'neurons': [32, 64, 128],
      'kernel_size': [(3, 3), (4, 4)],
      'pool_size': [(2, 2), (3, 3)]
    }

    best_accuracy = 0.0
    best_hyperparameters = {}
    for optimizer, neurons, kernel_size, pool_size in product(param_grid['optimizer'],
                                                         param_grid['neurons'],
                                                         param_grid['kernel_size'],
                                                         param_grid['pool_size']):
      model = create_model(neurons=neurons,
                         kernel_size=kernel_size,
                         pool_size=pool_size,
                         optimizer=optimizer)

      history = model.fit(train_images, train_labels, epochs=70,
                        validation_data=(test_images, test_labels))

      # plot and save the image
      # plt.figure(figsize=(8,6))
      # plt.plot(history.history['accuracy'], label='accuracy')
      # plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
      # plt.xlabel('Epoch')
      # plt.ylabel('Accuracy')
      # plt.ylim([0.5, 1.2])
      # plt.legend(loc='lower right')
      #save_fig(f"Training and validation accuracy for {img_size} x {img_size} at {date} with four classes")

      # Print the current hyperparameters
      print(f"Optimizer: {optimizer}, Neurons: {neurons}, Kernel Size: {kernel_size}, Pool Size: {pool_size}")

      test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

      if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_hyperparameters = {
            'optimizer': optimizer,
            'neurons': neurons,
            'kernel_size': kernel_size,
            'pool_size': pool_size
        }

      probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
      predictions = probability_model.predict(test_images)
      pred = np.argmax(predictions, axis=1)

      # # save the confusion matrix image
      # tf.math.confusion_matrix(test_labels, pred)
      # ConfusionMatrixDisplay.from_predictions(test_labels, pred)
      #save_fig(f"Confusion matrix for {img_size} x {img_size} at {date} with four classes")

      # save the precision and recall information to .txt files

      precision_recall_fscore_support(test_labels, pred, labels=[0, 1])
      #target_names = ['class 0', 'class 1', 'class 2', 'class 3']
      target_names = ['rainfed', 'fully irrigated', 'percent deficit', 'time delay']
      print(classification_report(test_labels, pred, target_names=target_names))
      report = classification_report(test_labels, pred, target_names=target_names)
      report_path = f"Precision and Recall for {img_size} x {img_size} at {date} with four classes.txt"
      text_file = open(report_path, "w")
      n = text_file.write(report)
      text_file.close()

        # Print the best hyperparameters and test accuracy
    print("\nBest Hyperparameters:")
    print(best_hyperparameters)
    print("Best Test Accuracy:", best_accuracy)